In [ ]:
# https://keras.io/examples/vision/video_classification/

### Importing and preparing dataframe

In [ ]:
!pip install -q git+https://github.com/tensorflow/docs

In [ ]:
!rm -rf sample_data

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
datapath = '/content/gdrive/MyDrive/data/'

In [ ]:
from tensorflow_docs.vis import embed
from tensorflow import keras
from imutils import paths

import matplotlib.pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
import imageio
import cv2
import os

In [ ]:
IMG_SIZE = 224
BATCH_SIZE = 64
EPOCHS = 10

MAX_SEQ_LENGTH = 20
NUM_FEATURES = 2048

In [ ]:
# open the file which have names of training videos
from os import listdir
from os.path import isfile,join

bc = [f for f in listdir(f"{datapath}goodtrain") if isfile(join(f"{datapath}goodtrain",f))]
sp = [f for f in listdir(f"{datapath}ShoulderPressGoodTrain") if isfile(join(f"{datapath}ShoulderPressGoodTrain",f))]
allvideos = bc+sp

videoclass = ['bicep_curl' for _ in range(len(bc))] + ['shoulder_press' for _ in range(len(sp))]

# creating a dataframe having video names
train = pd.DataFrame({'video_name':allvideos})
train['class'] = videoclass

print(train)

             video_name           class
0   20211024_221804.mp4      bicep_curl
1   20211024_221812.mp4      bicep_curl
2   20211024_221818.mp4      bicep_curl
3   20211024_221824.mp4      bicep_curl
4   20211024_221830.mp4      bicep_curl
..                  ...             ...
81  20211122_222039.mp4  shoulder_press
82  20211122_222028.mp4  shoulder_press
83  20211122_222033.mp4  shoulder_press
84  20211122_222021.mp4  shoulder_press
85  20211122_222105.mp4  shoulder_press

[86 rows x 2 columns]


In [ ]:
# open the .txt file which have names of test videos
from os import listdir
from os.path import isfile,join

bctest = [f for f in listdir(f"{datapath}goodtest") if isfile(join(f"{datapath}goodtest",f))]
sptest = [f for f in listdir(f"{datapath}ShoulderPressGoodTest") if isfile(join(f"{datapath}ShoulderPressGoodTest",f))]
alltest = bctest+sptest

testclass = ['bicep_curl' for _ in range(len(bctest))] + ['shoulder_press' for _ in range(len(sptest))]

# creating a dataframe having video names
test = pd.DataFrame({'video_name':alltest})
test['class'] = testclass

print(test)

       video_name           class
0      Good 2.mov      bicep_curl
1      Good 3.mp3      bicep_curl
2      Good 6.mov      bicep_curl
3      Good 7.mov      bicep_curl
4      Good 1.mov      bicep_curl
5      Good 8.mov      bicep_curl
6      Good 9.mov      bicep_curl
7      Good 4.mov      bicep_curl
8     Good 10.mov      bicep_curl
9     Good 12.mov      bicep_curl
10    Good 11.mov      bicep_curl
11    Good 13.mov      bicep_curl
12  SP good 1.mov  shoulder_press
13  SP good 2.mov  shoulder_press
14  SP good 3.mov  shoulder_press


### Model Training

In [ ]:
def load_video(path, i, max_frames=0):
    if path == "train":
        videoname = train['video_name'][i]
        videoclass = train['class'][i]
        if videoclass == "bicep_curl":
          videotype = "goodT"
        elif videoclass == "shoulder_press":
          videotype = "ShoulderPressGoodT"
        video = datapath+videotype+"rain/"+videoname

    elif path == "test":
        videoname = test['video_name'][i]
        videoclass = test['class'][i]
        if videoclass == "bicep_curl":
          videotype = "goodT"
        elif videoclass == "shoulder_press":
          videotype = "ShoulderPressGoodT"
        video = datapath+videotype+"est/"+videoname
    cap = cv2.VideoCapture(video)
    frames = []
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frameID = cap.get(1)
            if (frameID%2==0):
                frame = cv2.resize(frame, (224, 224))
                frames.append(frame)
            # storing the frames in a new folder named train_1
            # filename = f"{path}_1/{train['video_name'][i]}_frame{frameID/2}_{train['class'][i]}.jpg"
            # cv2.imwrite(filename, frame)
            # if len(frames) == max_frames:
            #     break
    finally:
        cap.release()
    return np.array(frames)

In [ ]:
def build_feature_extractor():
    feature_extractor = keras.applications.InceptionV3(
        weights="imagenet",
        include_top=False,
        pooling="avg",
        input_shape=(IMG_SIZE, IMG_SIZE, 3),
    )
    preprocess_input = keras.applications.inception_v3.preprocess_input

    inputs = keras.Input((IMG_SIZE, IMG_SIZE, 3))
    preprocessed = preprocess_input(inputs)

    outputs = feature_extractor(preprocessed)
    return keras.Model(inputs, outputs, name="feature_extractor")


feature_extractor = build_feature_extractor()


87924736/87910968 [==============================] - 1s 0us/step


In [ ]:
label_processor = keras.layers.StringLookup(
    num_oov_indices=0, vocabulary=np.unique(train['class'])
)
print(label_processor.get_vocabulary())

['bicep_curl', 'shoulder_press']


In [ ]:
def prepare_all_videos(df, root_dir):
    num_samples = len(df)
    video_paths = df["video_name"].values.tolist()
    labels = df["class"].values
    labels = label_processor(labels[..., None]).numpy()

    # `frame_masks` and `frame_features` are what we will feed to our sequence model.
    # `frame_masks` will contain a bunch of booleans denoting if a timestep is
    # masked with padding or not.
    frame_masks = np.zeros(shape=(num_samples, MAX_SEQ_LENGTH), dtype="bool")
    frame_features = np.zeros(
        shape=(num_samples, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
    )

    # For each video.
    for idx, path in enumerate(video_paths):
        # Gather all its frames and add a batch dimension.
        # print(os.path.join(datapath, "bad"+ root_dir, path))
        frames = load_video(root_dir, idx)
        # frames2 = load_video(os.path.join(datapath, "bad"+root_dir, path), idx)
        frames = frames[None, ...]
        # print(frames)

        # Initialize placeholders to store the masks and features of the current video.
        temp_frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
        temp_frame_features = np.zeros(
            shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32"
        )

        # Extract features from the frames of the current video.
        for i, batch in enumerate(frames):
            video_length = batch.shape[0]
            length = min(MAX_SEQ_LENGTH, video_length)
            for j in range(length):
                temp_frame_features[i, j, :] = feature_extractor.predict(
                    batch[None, j, :]
                )
            temp_frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

        frame_features[idx,] = temp_frame_features.squeeze()
        frame_masks[idx,] = temp_frame_mask.squeeze()

    return (frame_features, frame_masks), labels


train_data, train_labels = prepare_all_videos(train, "train")
test_data, test_labels = prepare_all_videos(test, "test")

print(f"Frame features in train set: {train_data[0].shape}")
print(f"Frame masks in train set: {train_data[1].shape}")

Frame features in train set: (86, 20, 2048)
Frame masks in train set: (86, 20)


In [ ]:
# Utility for our sequence model.
def get_sequence_model():
    class_vocab = label_processor.get_vocabulary()

    frame_features_input = keras.Input((MAX_SEQ_LENGTH, NUM_FEATURES))
    mask_input = keras.Input((MAX_SEQ_LENGTH,), dtype="bool")

    # Refer to the following tutorial to understand the significance of using `mask`:
    # https://keras.io/api/layers/recurrent_layers/gru/
    x = keras.layers.GRU(16, return_sequences=True)(
        frame_features_input, mask=mask_input
    )
    x = keras.layers.GRU(8)(x)
    x = keras.layers.Dropout(0.4)(x)
    x = keras.layers.Dense(8, activation="relu")(x)
    output = keras.layers.Dense(len(class_vocab), activation="softmax")(x)

    rnn_model = keras.Model([frame_features_input, mask_input], output)

    rnn_model.compile(
        loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"]
    )
    return rnn_model


# Utility for running experiments.
def run_experiment():
    filepath = "/tmp/video_classifier"
    checkpoint = keras.callbacks.ModelCheckpoint(
        filepath, save_weights_only=True, save_best_only=True, verbose=1
    )

    seq_model = get_sequence_model()
    history = seq_model.fit(
        [train_data[0], train_data[1]],
        train_labels,
        validation_split=0.3,
        epochs=EPOCHS,
        callbacks=[checkpoint],
    )

    seq_model.load_weights(filepath)
    _, accuracy = seq_model.evaluate([test_data[0], test_data[1]], test_labels)
    print(f"Test accuracy: {round(accuracy * 100, 2)}%")

    return history, seq_model


_, sequence_model = run_experiment()


Epoch 1/10
1/2 [==============>...............] - ETA: 7s - loss: 0.5909 - accuracy: 0.9688
Epoch 00001: val_loss improved from inf to 0.10581, saving model to /tmp/video_classifier
2/2 [==============================] - 10s 3s/step - loss: 0.5609 - accuracy: 0.7000 - val_loss: 0.1058 - val_accuracy: 1.0000
Epoch 2/10
1/2 [==============>...............] - ETA: 0s - loss: 0.4814 - accuracy: 0.9688
Epoch 00002: val_loss improved from 0.10581 to 0.08955, saving model to /tmp/video_classifier
2/2 [==============================] - 0s 92ms/step - loss: 0.4793 - accuracy: 0.9833 - val_loss: 0.0895 - val_accuracy: 1.0000
Epoch 3/10
1/2 [==============>...............] - ETA: 0s - loss: 0.4850 - accuracy: 1.0000
Epoch 00003: val_loss improved from 0.08955 to 0.08285, saving model to /tmp/video_classifier
2/2 [==============================] - 0s 95ms/step - loss: 0.4583 - accuracy: 1.0000 - val_loss: 0.0828 - val_accuracy: 1.0000
Epoch 4/10
1/2 [==============>...............] - ETA: 0s - los

### Model Evaluation

In [ ]:
def prepare_single_video(frames):
    frames = frames[None, ...]
    frame_mask = np.zeros(shape=(1, MAX_SEQ_LENGTH,), dtype="bool")
    frame_features = np.zeros(shape=(1, MAX_SEQ_LENGTH, NUM_FEATURES), dtype="float32")

    for i, batch in enumerate(frames):
        video_length = batch.shape[0]
        length = min(MAX_SEQ_LENGTH, video_length)
        for j in range(length):
            frame_features[i, j, :] = feature_extractor.predict(batch[None, j, :])
        frame_mask[i, :length] = 1  # 1 = not masked, 0 = masked

    return frame_features, frame_mask


def sequence_prediction(i):
    class_vocab = label_processor.get_vocabulary()

    frames = load_video("test", i)
    frame_features, frame_mask = prepare_single_video(frames)
    probabilities = sequence_model.predict([frame_features, frame_mask])[0]

    for i in np.argsort(probabilities)[::-1]:
        print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
    return frames


for idx, video in enumerate(test["video_name"].values.tolist()):
    print(f"Test video path: {video}")
    test_frames = sequence_prediction(idx)

Test video path: Good 2.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 3.mp3
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 6.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 7.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 1.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 8.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 9.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 4.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 10.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 12.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 11.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: Good 13.mov
  bicep_curl: 51.00%
  shoulder_press: 49.00%
Test video path: SP good 1.mov
  shoulder_press: 95.16%
  bicep_curl:  4.84%
Test video path: SP good 2.mov


In [ ]:
# def sequence_prediction(path, i):
#     class_vocab = label_processor.get_vocabulary()

#     # frames = load_video(os.path.join("test", path), i)
#     test_data, test_labels = prepare_all_videos(test, "test")
#     probabilities = sequence_model.predict(test_data)[0]

#     for i in np.argsort(probabilities)[::-1]:
#         print(f"  {class_vocab[i]}: {probabilities[i] * 100:5.2f}%")
#     # return frames

# for idx, video in enumerate(test["video_name"].values.tolist()):
#     print(f"Test video path: {video}")
#     test_frames = sequence_prediction(video, idx)
